In [5]:
import json


def evaluate_result(generation_result_path: str, system_prompt: str, openai_client):
    # Construct prompt
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": json.dumps(open(generation_result_path).read())}
    ]

    # Generate a completion
    completion = openai_client.chat.completions.create(
        # model="gpt-4-turbo",
        model="gpt-4o",
        response_format={"type": "json_object"},
        messages=messages
    )

    # Get the generated text
    result_string = completion.choices[0].message.content

    # Convert the generated text into a dictionary
    evaluation = json.loads(result_string)

    # Return the dictionary
    return evaluation

In [6]:
import os
import dotenv
from openai import OpenAI


# Load the environment variables
dotenv.load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

# Create an instance of the OpenAI class
openai_api = OpenAI()

# Read the system prompt
with open('prompt.txt', 'r') as file:
    # Read the entire content of the file
    sys_prompt = file.read()

# Test tagging the education dictionary
result = evaluate_result(
    os.getenv('Test_FILE_PATH'),
    sys_prompt, 
    openai_api)

# Print the result
print(result)

{'valid_json': True, 'accuracy_rating': 8, 'assumption_rating': 7, 'justification_rating': 8, 'missing_info_rating': 6, 'over_generalization_rating': 5, 'confidence_score': 8, 'comments': "The generated JSON follows the required structure and is valid. The inferred industry and profession are accurate based on the company name and job title. The skills listed are relevant to a Mechanical Engineer role within a manufacturing or automation context, which is reasonable for Premier Automation. However, some skills like 'development' and 'automation' are slightly broad without more context. The tags are mostly relevant, but 'Pittsburgh area' might be considered too generic. Overall, the output is solid but could improve specificity in both skills and tags."}


In [13]:
import json
import os
import dotenv
from openai import OpenAI


# Load the environment variables
dotenv.load_dotenv()

# Create an instance of the OpenAI class
openai_api = OpenAI()

# Directory containing all files to process
results_directory = os.getenv('TAGS_FOLDER_PATH') + 'outputs_json/'

# Read the system prompt
with open('prompt.txt', 'r') as file:
    sys_prompt = file.read()

# List to hold all the results
all_results = []

# Iterate over all files in the results directory
for filename in os.listdir(results_directory):
    if filename.endswith(".json"):  # Adjust the file type if necessary
        file_path = os.path.join(results_directory, filename)
        # Evaluate the result for each file
        result = evaluate_result(file_path, sys_prompt, openai_api)
        # Append the result to the list
        all_results.append(result)

In [14]:
import json
import pandas as pd


# Print the number of results
print(len(all_results))

# Save the results to a JSON file
output_path = f'evaluation_{len(all_results)}_gpt_4o.json'
with open(output_path, 'w') as json_file:
    json.dump(all_results, json_file, indent=4)

# Save the results to a CSV file
eval_df = pd.DataFrame(all_results)
eval_df.to_csv(f'evaluation_{len(all_results)}_gpt_4o.csv', index=False)

600
